In [12]:
import numpy as np
from PIL import Image

In [13]:
# to randomize and shuffel the lists indices
import random
random.seed(5)

In [14]:
total_images = 555
number_of_train_images = 500
number_of_test_images = 55
indices = ["%04d" % x for x in range(total_images)]
n = 256

In [15]:
TP = []
TN = []
FP = []
FN = []

In [16]:
# we will do crossfold validation
# we will 10 fold and will do 90% for training and 10% for testing

for i in range(1):
    random.shuffle(indices)
    track_image = np.zeros(shape=(total_images))

    # first train the training set
    skinPixel = np.zeros(shape=(n, n, n))
    nonSkinPixel = np.zeros(shape=(n, n, n))
    skinPixelSum = 0
    nonSkinPixelSum = 0
    skinProbability = np.zeros(shape=(n, n, n))
    nonSkinProbability = np.zeros(shape=(n, n, n))
    learning_ratio = np.zeros(shape=(n, n, n)) 

    for index in range(number_of_train_images):
        track_image[int(indices[index])] = 1 

        mask_im = Image.open("/home/asif/DBMS-II/SkinDetection/Mask/"+str(indices[index])+".bmp")
        unmask_im = Image.open("/home/asif/DBMS-II/SkinDetection/NonMask/"+str(indices[index])+".jpg")
        mask_pix = mask_im.load()
        unmask_pix = unmask_im.load()

        width, height = mask_im.size

        for x in range(width):
            for y in range(height):
                red, green, blue = mask_pix[x, y]

                if blue > 250 and green > 250 and red > 250:  
                    red, green, blue = unmask_pix[x, y]
                    nonSkinPixel[red, green, blue] += 1
                    nonSkinPixelSum += 1
                else:
                    skinPixel[red, green, blue] += 1
                    skinPixelSum += 1

        print("Image ", index, " - DONE!")
    
    # now calculate the probabilities
    for r in range(n):
        for g in range(n):
            for b in range(n):
                skinProbability[r, g, b] = skinPixel[r, g, b]/skinPixelSum
                nonSkinProbability[r, g, b] = nonSkinPixel[r, g, b]/nonSkinPixelSum

                if (nonSkinPixel[r, g, b] != 0):
                    learning_ratio[r, g, b] = skinProbability[r, g, b] / nonSkinProbability[r, g, b]
                else:
                    learning_ratio[r, g, b] = 0.0

    print("Training "+str(i)+" done")

    # now test
    T = 0.4
    true_skin = 0
    true_non_skin = 0
    false_skin = 0
    false_non_skin = 0
   
    for index in indices:
        if track_image[int(index)] != 1:
            test_non_mask_image_path = "/home/asif/DBMS-II/SkinDetection/NonMask/"+str(index)+".jpg"
            test_non_mask_im = Image.open(test_non_mask_image_path)
            test_non_mask_pix = test_non_mask_im.load()
            width, height = test_non_mask_im.size

            test_mask_image_path = "/home/asif/DBMS-II/SkinDetection/Mask/"+str(index)+".bmp"
            test_mask_im = Image.open(test_mask_image_path)
            test_mask_pix = test_mask_im.load()

            for x in range(width):
                for y in range(height):
                    r, g, b = test_non_mask_pix[x, y]   
                    
                    red, green, blue = test_mask_pix[x, y]

                    if abs(learning_ratio[r][g][b])<T:  # non skin
                        if red<250 or green<250 or blue<250:
                            false_non_skin = false_non_skin + 1
                        else:
                            true_non_skin = true_non_skin + 1
                    else: # skin
                        if  red>250 and green>250 and blue>250:
                            false_skin = false_skin + 1
                        else:
                            true_skin = true_skin + 1
        

    #print
    print(true_skin, false_skin, true_non_skin, false_non_skin)
    # now store the results in list
    TP.append(true_skin)
    TN.append(true_non_skin)
    FP.append(false_skin)
    FN.append(false_non_skin)

    print("Testing "+str(i)+" done")


Image  0  - DONE!
Image  1  - DONE!
Image  2  - DONE!
Image  3  - DONE!
Image  4  - DONE!
Image  5  - DONE!
Image  6  - DONE!
Image  7  - DONE!
Image  8  - DONE!
Image  9  - DONE!
Image  10  - DONE!
Image  11  - DONE!
Image  12  - DONE!
Image  13  - DONE!
Image  14  - DONE!
Image  15  - DONE!
Image  16  - DONE!
Image  17  - DONE!
Image  18  - DONE!
Image  19  - DONE!
Image  20  - DONE!
Image  21  - DONE!
Image  22  - DONE!
Image  23  - DONE!
Image  24  - DONE!
Image  25  - DONE!
Image  26  - DONE!
Image  27  - DONE!
Image  28  - DONE!
Image  29  - DONE!
Image  30  - DONE!
Image  31  - DONE!
Image  32  - DONE!
Image  33  - DONE!
Image  34  - DONE!
Image  35  - DONE!
Image  36  - DONE!
Image  37  - DONE!
Image  38  - DONE!
Image  39  - DONE!
Image  40  - DONE!
Image  41  - DONE!
Image  42  - DONE!
Image  43  - DONE!
Image  44  - DONE!
Image  45  - DONE!
Image  46  - DONE!
Image  47  - DONE!
Image  48  - DONE!
Image  49  - DONE!
Image  50  - DONE!
Image  51  - DONE!
Image  52  - DONE!
Ima

In [17]:
# Now store the results in file
with open('Outputs/TP.txt', 'w') as filehandle:
    for listitem in TP:
        filehandle.write(f'{listitem}\n')

with open('Outputs/TN.txt', 'w') as filehandle:
    for listitem in TN:
        filehandle.write(f'{listitem}\n')

with open('Outputs/FP.txt', 'w') as filehandle:
    for listitem in FP:
        filehandle.write(f'{listitem}\n')

with open('Outputs/FN.txt', 'w') as filehandle:
    for listitem in FN:
        filehandle.write(f'{listitem}\n')